In [1]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')

from fastdownload import download_url

from fastai.vision.all import *
import timm
import numpy as np
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd
import pickle

## Getting lists of initial pictures

In [6]:
page_t = requests.get('https://boardgamegeek.com/browse/boardgame')
soup_t = bs4(page_t.content, 'html.parser')

In [17]:
links=list(soup_t.find_all("img", src=re.compile(r'geekdo-images')))

In [23]:
str(links[0])

'<img alt="Board Game: Gloomhaven" src="https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__micro/img/sQyh47ClBO3d5sxPm73hMvM-JV4=/fit-in/64x64/filters:strip_icc()/pic2437871.jpg"/>'

In [25]:
patt= re.compile(r'pic\d*')

In [35]:
gns=[]
for link in links :
    gns.append(patt.findall(str(link))[0][3:])

In [38]:
type(gns[0])

str

In [39]:
gns[0].isnumeric()

True

In [48]:
type(int(gns[0]) )==int

True

In [52]:
with open('game_numbers', 'wb') as fp:
    pickle.dump(gns, fp)

## Working with the dataframe of game_links

In [12]:
df=pd.read_csv('game_links_selenium.csv',header=None,index_col=0)

In [13]:
df.drop(['crew-mission-deep-sea','pandemic-legacy-season-2'],axis=0,inplace=True)

In [14]:
df2=pd.DataFrame(df.stack())

In [15]:
df2.columns=['link']

In [16]:
df2=df2.reset_index().drop('level_1',axis=1)

In [17]:
df2.columns=['game_name','image_page']

In [18]:
df2['image_number']=(df2['image_page'].str.split('/')).map(lambda l : l[-2])

In [19]:
df2['direct_link']= None

Idea: Direct link can be filled while downloading for possible future use.

In [10]:
df2[df2['game_name']=='food-chain-magnate']

,game_name,image_page,image_number,direct_link
5134,food-chain-magnate,https://boardgamegeek.com/image/2649434/food-chain-magnate,2649434,None
5135,food-chain-magnate,https://boardgamegeek.com/image/2649623/food-chain-magnate,2649623,None
5136,food-chain-magnate,https://boardgamegeek.com/image/2652009/food-chain-magnate,2652009,None
5137,food-chain-magnate,https://boardgamegeek.com/image/2654169/food-chain-magnate,2654169,None
5138,food-chain-magnate,https://boardgamegeek.com/image/2659708/food-chain-magnate,2659708,None
...,...,...,...,...
5280,food-chain-magnate,https://boardgamegeek.com/image/2933438/food-chain-magnate,2933438,None
5281,food-chain-magnate,https://boardgamegeek.com/image/2935379/food-chain-magnate,2935379,None
5282,food-chain-magnate,https://boardgamegeek.com/image/2941651/food-chain-magnate,2941651,None
5283,food-chain-magnate,https://boardgamegeek.com/image/2941654/food-chain-magnate,2941654,None


In [19]:
len(df2)-5134

2114

In [18]:
df2.tail(2114)

,game_name,image_page,image_number,direct_link
5134,food-chain-magnate,https://boardgamegeek.com/image/2649434/food-chain-magnate,2649434,None
5135,food-chain-magnate,https://boardgamegeek.com/image/2649623/food-chain-magnate,2649623,None
5136,food-chain-magnate,https://boardgamegeek.com/image/2652009/food-chain-magnate,2652009,None
5137,food-chain-magnate,https://boardgamegeek.com/image/2654169/food-chain-magnate,2654169,None
5138,food-chain-magnate,https://boardgamegeek.com/image/2659708/food-chain-magnate,2659708,None
...,...,...,...,...
7243,crew-quest-planet-nine,https://boardgamegeek.com/image/6091311/crew-quest-planet-nine,6091311,None
7244,crew-quest-planet-nine,https://boardgamegeek.com/image/6091312/crew-quest-planet-nine,6091312,None
7245,crew-quest-planet-nine,https://boardgamegeek.com/image/6091313/crew-quest-planet-nine,6091313,None
7246,crew-quest-planet-nine,https://boardgamegeek.com/image/6091314/crew-quest-planet-nine,6091314,None


In [20]:
pd.to_pickle(df2,'df_images')

In [21]:
pd.to_pickle(df2.head(),'df_images_head')

In [22]:
pd.to_pickle(df2.tail(2114),'df_images_tail')

# Testing

### Testing

In [28]:
soup_t.find_all("a",src=re.compile(r'geekdo-images'))

[]

In [2]:
page_t = requests.get('https://boardgamegeek.com/image/2669798')

In [3]:
soup_t = bs4(page_t.content, 'html.parser')

In [4]:
def extract_img(image_number): 
    page = requests.get('https://boardgamegeek.com/image/'+str(image_number))
    soup = bs4(page.content, 'html.parser')
    uncleaned_image_link=(str(soup.find('link',href=re.compile(r'geekdo-images'))))
    patt= re.compile(r'https.*'+str(image_number)+r'[^"]*')
    image_link=patt.findall(uncleaned_image_link) #add an assertion that the list is nonempty
    
    uncleaned_game_name=(str(soup.find('link',rel='canonical')))
    patt2= re.compile(str(image_number)+r'/[^"]*')
    game_name=patt2.findall(uncleaned_game_name) #add an assertion that the list is nonempty
    if len(image_link)==0 or len(game_name)==0:
        return 'invalid', uncleaned_image_link +'\n' + uncleaned_game_name +'\n' +str(soup)
    return game_name[0][8:], image_link[0]  #,page, soup, uncleaned_image_link, patt,

In [9]:
def get_link(image_number):
    page = requests.get('https://boardgamegeek.com/image/'+str(image_number))
    if page.status_code==404:
        game_name='404'
        image_link=None
    else:
        soup = bs4(page.content, 'html.parser')
        uncleaned_image_link=(str(soup.find('link',href=re.compile(r'geekdo-images')))) #isolates the full link element to the image
        patt= re.compile(r'https.*'+str(image_number)+r'[^"]*')  
        image_link=patt.findall(uncleaned_image_link) #removes everything except the link iteself

        uncleaned_game_name=(str(soup.find('link',rel='canonical'))) 
        patt2= re.compile(str(image_number)+r'/[^"]*')
        game_name=patt2.findall(uncleaned_game_name)

        if image_link==[] or game_name==[]:
            game_name=str(page.status_code)
            image_link= None
        else: 
            game_name=game_name[0][8:]
            image_link=image_link[0]

    return np.asarray([image_number,game_name,image_link])

In [5]:
page_t = requests.get('https://boardgamegeek.com/image/'+str(2669812))
soup_t = bs4(page_t.content, 'html.parser')
uncleaned_image_link_t=(str(soup_t.find('link',href=re.compile(r'geekdo-images'))))
patt_t= re.compile(r'https.*'+str(2669812)+r'[^"]*')
image_link_t=patt_t.findall(uncleaned_image_link_t)

In [6]:
page_t.status_code==404

False

In [10]:
extract_img(2669798)

('gloomhaven',
 'https://cf.geekdo-images.com/HNMHfQ49oxs8ig-IEmNQWQ__imagepage/img/nNo4l2NjXToU3DCoBnMIbDExJu0=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669798.png')

In [11]:
extract_img(2669900)

('jmaydet',
 'https://cf.geekdo-images.com/0WHEngboBp96fNQmqVNjiQ__imagepage/img/95blS1S21dYclnS_bMi7ws_pRq8=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669900.jpg')

### Testing for: "Working with the dataframe of game_links"

Need to create a dataframe with rows: individual game links using columns=[game_name, image_number, image_url] using pivot? No, pivot doesn't seem right. TBD

In [49]:
df=pd.read_csv('game_links_selenium.csv',header=None,index_col=0)

In [50]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,142,143,144,145,146,147,148,149,150,151
0,,,,,,,,,,,,,,,,,,,,,
gloomhaven,https://boardgamegeek.com/image/2437871/gloomhaven,https://boardgamegeek.com/image/2556210/gloomhaven,https://boardgamegeek.com/image/2563554/gloomhaven,https://boardgamegeek.com/image/2571642/gloomhaven,https://boardgamegeek.com/image/2579136/gloomhaven,https://boardgamegeek.com/image/2586084/gloomhaven,https://boardgamegeek.com/image/2594721/gloomhaven,https://boardgamegeek.com/image/2602482/gloomhaven,https://boardgamegeek.com/image/2632373/gloomhaven,https://boardgamegeek.com/image/2632375/gloomhaven,...,https://boardgamegeek.com/image/3392632/gloomhaven,https://boardgamegeek.com/image/3392633/gloomhaven,https://boardgamegeek.com/image/3392634/gloomhaven,https://boardgamegeek.com/image/3392635/gloomhaven,https://boardgamegeek.com/image/3392636/gloomhaven,https://boardgamegeek.com/image/3392664/gloomhaven,https://boardgamegeek.com/image/3392667/gloomhaven,https://boardgamegeek.com/image/3392669/gloomhaven,https://boardgamegeek.com/image/3392671/gloomhaven,https://boardgamegeek.com/image/3392672/gloomhaven
brass-birmingham,https://boardgamegeek.com/image/3490053/brass-birmingham,https://boardgamegeek.com/image/3490095/brass-birmingham,https://boardgamegeek.com/image/3500869/brass-birmingham,https://boardgamegeek.com/image/3509697/brass-birmingham,https://boardgamegeek.com/image/3509698/brass-birmingham,https://boardgamegeek.com/image/3549793/brass-birmingham,https://boardgamegeek.com/image/3733951/brass-birmingham,https://boardgamegeek.com/image/3733952/brass-birmingham,https://boardgamegeek.com/image/4099876/brass-birmingham,https://boardgamegeek.com/image/4099877/brass-birmingham,...,https://boardgamegeek.com/image/4310169/brass-birmingham,https://boardgamegeek.com/image/4311986/brass-birmingham,https://boardgamegeek.com/image/4316215/brass-birmingham,https://boardgamegeek.com/image/4317265/brass-birmingham,https://boardgamegeek.com/image/4325228/brass-birmingham,https://boardgamegeek.com/image/4325229/brass-birmingham,https://boardgamegeek.com/image/4325230/brass-birmingham,https://boardgamegeek.com/image/4325231/brass-birmingham,https://boardgamegeek.com/image/4341198/brass-birmingham,https://boardgamegeek.com/image/4346280/brass-birmingham
pandemic-legacy-season-1,https://boardgamegeek.com/image/2452831/pandemic-legacy-season-1,https://boardgamegeek.com/image/2474195/pandemic-legacy-season-1,https://boardgamegeek.com/image/2474196/pandemic-legacy-season-1,https://boardgamegeek.com/image/2488668/pandemic-legacy-season-1,https://boardgamegeek.com/image/2488706/pandemic-legacy-season-1,https://boardgamegeek.com/image/2620676/pandemic-legacy-season-1,https://boardgamegeek.com/image/2649655/pandemic-legacy-season-1,https://boardgamegeek.com/image/2649656/pandemic-legacy-season-1,https://boardgamegeek.com/image/2649657/pandemic-legacy-season-1,https://boardgamegeek.com/image/2649659/pandemic-legacy-season-1,...,https://boardgamegeek.com/image/2807432/pandemic-legacy-season-1,https://boardgamegeek.com/image/2808169/pandemic-legacy-season-1,https://boardgamegeek.com/image/2809485/pandemic-legacy-season-1,https://boardgamegeek.com/image/2809486/pandemic-legacy-season-1,https://boardgamegeek.com/image/2809817/pandemic-legacy-season-1,https://boardgamegeek.com/image/2813424/pandemic-legacy-season-1,https://boardgamegeek.com/image/2814770/pandemic-legacy-season-1,https://boardgamegeek.com/image/2817971/pandemic-legacy-season-1,https://boardgamegeek.com/image/2817972/pandemic-legacy-season-1,https://boardgamegeek.com/image/2817974/pandemic-legacy-season-1
ark-nova,https://boardgamegeek.com/image/6293412/ark-nova,https://boardgamegeek.com/image/6294036/ark-nova,https://boardgamegeek.com/image/6294513/ark-nova,https://boardgamegeek.com/image/6398292/ark-nova,https://boardgamegeek.com/image/6417715/ark-nova,https://boardgamegeek.com/image/6428595/ark-nova,https://boardgamegeek.com/image/6446792/ark-nova,https://boardgamegeek.com/image/64

In [51]:
df.drop(['crew-mission-deep-sea','pandemic-legacy-season-2'],axis=0,inplace=True)

In [52]:
df.isnull().sum(axis=1).sum()

0

In [53]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,142,143,144,145,146,147,148,149,150,151
0,,,,,,,,,,,,,,,,,,,,,
gloomhaven,https://boardgamegeek.com/image/2437871/gloomhaven,https://boardgamegeek.com/image/2556210/gloomhaven,https://boardgamegeek.com/image/2563554/gloomhaven,https://boardgamegeek.com/image/2571642/gloomhaven,https://boardgamegeek.com/image/2579136/gloomhaven,https://boardgamegeek.com/image/2586084/gloomhaven,https://boardgamegeek.com/image/2594721/gloomhaven,https://boardgamegeek.com/image/2602482/gloomhaven,https://boardgamegeek.com/image/2632373/gloomhaven,https://boardgamegeek.com/image/2632375/gloomhaven,...,https://boardgamegeek.com/image/3392632/gloomhaven,https://boardgamegeek.com/image/3392633/gloomhaven,https://boardgamegeek.com/image/3392634/gloomhaven,https://boardgamegeek.com/image/3392635/gloomhaven,https://boardgamegeek.com/image/3392636/gloomhaven,https://boardgamegeek.com/image/3392664/gloomhaven,https://boardgamegeek.com/image/3392667/gloomhaven,https://boardgamegeek.com/image/3392669/gloomhaven,https://boardgamegeek.com/image/3392671/gloomhaven,https://boardgamegeek.com/image/3392672/gloomhaven
brass-birmingham,https://boardgamegeek.com/image/3490053/brass-birmingham,https://boardgamegeek.com/image/3490095/brass-birmingham,https://boardgamegeek.com/image/3500869/brass-birmingham,https://boardgamegeek.com/image/3509697/brass-birmingham,https://boardgamegeek.com/image/3509698/brass-birmingham,https://boardgamegeek.com/image/3549793/brass-birmingham,https://boardgamegeek.com/image/3733951/brass-birmingham,https://boardgamegeek.com/image/3733952/brass-birmingham,https://boardgamegeek.com/image/4099876/brass-birmingham,https://boardgamegeek.com/image/4099877/brass-birmingham,...,https://boardgamegeek.com/image/4310169/brass-birmingham,https://boardgamegeek.com/image/4311986/brass-birmingham,https://boardgamegeek.com/image/4316215/brass-birmingham,https://boardgamegeek.com/image/4317265/brass-birmingham,https://boardgamegeek.com/image/4325228/brass-birmingham,https://boardgamegeek.com/image/4325229/brass-birmingham,https://boardgamegeek.com/image/4325230/brass-birmingham,https://boardgamegeek.com/image/4325231/brass-birmingham,https://boardgamegeek.com/image/4341198/brass-birmingham,https://boardgamegeek.com/image/4346280/brass-birmingham
pandemic-legacy-season-1,https://boardgamegeek.com/image/2452831/pandemic-legacy-season-1,https://boardgamegeek.com/image/2474195/pandemic-legacy-season-1,https://boardgamegeek.com/image/2474196/pandemic-legacy-season-1,https://boardgamegeek.com/image/2488668/pandemic-legacy-season-1,https://boardgamegeek.com/image/2488706/pandemic-legacy-season-1,https://boardgamegeek.com/image/2620676/pandemic-legacy-season-1,https://boardgamegeek.com/image/2649655/pandemic-legacy-season-1,https://boardgamegeek.com/image/2649656/pandemic-legacy-season-1,https://boardgamegeek.com/image/2649657/pandemic-legacy-season-1,https://boardgamegeek.com/image/2649659/pandemic-legacy-season-1,...,https://boardgamegeek.com/image/2807432/pandemic-legacy-season-1,https://boardgamegeek.com/image/2808169/pandemic-legacy-season-1,https://boardgamegeek.com/image/2809485/pandemic-legacy-season-1,https://boardgamegeek.com/image/2809486/pandemic-legacy-season-1,https://boardgamegeek.com/image/2809817/pandemic-legacy-season-1,https://boardgamegeek.com/image/2813424/pandemic-legacy-season-1,https://boardgamegeek.com/image/2814770/pandemic-legacy-season-1,https://boardgamegeek.com/image/2817971/pandemic-legacy-season-1,https://boardgamegeek.com/image/2817972/pandemic-legacy-season-1,https://boardgamegeek.com/image/2817974/pandemic-legacy-season-1
ark-nova,https://boardgamegeek.com/image/6293412/ark-nova,https://boardgamegeek.com/image/6294036/ark-nova,https://boardgamegeek.com/image/6294513/ark-nova,https://boardgamegeek.com/image/6398292/ark-nova,https://boardgamegeek.com/image/6417715/ark-nova,https://boardgamegeek.com/image/6428595/ark-nova,https://boardgamegeek.com/image/6446792/ark-nova,https://boardgamegeek.com/image/64

In [54]:
df.loc['gloomhaven']

1      https://boardgamegeek.com/image/2437871/gloomhaven
2      https://boardgamegeek.com/image/2556210/gloomhaven
3      https://boardgamegeek.com/image/2563554/gloomhaven
4      https://boardgamegeek.com/image/2571642/gloomhaven
5      https://boardgamegeek.com/image/2579136/gloomhaven
                              ...                        
147    https://boardgamegeek.com/image/3392664/gloomhaven
148    https://boardgamegeek.com/image/3392667/gloomhaven
149    https://boardgamegeek.com/image/3392669/gloomhaven
150    https://boardgamegeek.com/image/3392671/gloomhaven
151    https://boardgamegeek.com/image/3392672/gloomhaven
Name: gloomhaven, Length: 151, dtype: object

In [30]:
#df=df.transpose()

In [67]:
df2=pd.DataFrame(df.stack())

In [68]:
df2.head()

0
0                                                               
gloomhaven 1  https://boardgamegeek.com/image/2437871/gloomhaven
           2  https://boardgamegeek.com/image/2556210/gloomhaven
           3  https://boardgamegeek.com/image/2563554/gloomhaven
           4  https://boardgamegeek.com/image/2571642/gloomhaven
           5  https://boardgamegeek.com/image/2579136/gloomhaven

In [69]:
df2.columns=['link']

In [70]:
df2=df2.reset_index().drop('level_1',axis=1)

In [71]:
df2.columns=['game_name','image_page']

In [80]:
df2['image_number']=(df2['image_page'].str.split('/')).map(lambda l : l[-2])

In [84]:
df2.set_index('image_number',inplace=True)

In [87]:
df2=df2.reset_index()

In [90]:
df2

,image_number,game_name,image_page
0,2437871,gloomhaven,https://boardgamegeek.com/image/2437871/gloomhaven
1,2556210,gloomhaven,https://boardgamegeek.com/image/2556210/gloomhaven
2,2563554,gloomhaven,https://boardgamegeek.com/image/2563554/gloomhaven
3,2571642,gloomhaven,https://boardgamegeek.com/image/2571642/gloomhaven
4,2579136,gloomhaven,https://boardgamegeek.com/image/2579136/gloomhaven
...,...,...,...
7243,6091311,crew-quest-planet-nine,https://boardgamegeek.com/image/6091311/crew-quest-planet-nine
7244,6091312,crew-quest-planet-nine,https://boardgamegeek.com/image/6091312/crew-quest-planet-nine
7245,6091313,crew-quest-planet-nine,https://boardgamegeek.com/image/6091313/crew-quest-planet-nine
7246,6091314,crew-quest-planet-nine,https://boardgamegeek.com/image/6091314/crew-quest-planet-nine


In [91]:
df2['direct_link']= None

Idea: Direct link can be filled while downloading for possible future use.

In [129]:
pd.to_pickle(df2,'df_images')

In [130]:
pd.to_pickle(df2.head(),'df_images_head')

In [105]:
url=df2.iloc[0]
output_folder='boardgame-pictures'
patt= re.compile(r'https[^"]*')  

In [98]:
url

image_number                                               2437871
game_name                                               gloomhaven
image_page      https://boardgamegeek.com/image/2437871/gloomhaven
direct_link                                                   None
Name: 0, dtype: object

In [101]:
page = requests.get(url.image_page)

In [102]:
soup = bs4(page.content, 'html.parser')

In [103]:
uncleaned_image_link=(str(soup.find('link',href=re.compile(r'geekdo-images')))) 

In [106]:
dl=patt.findall(uncleaned_image_link) #removes everything except the link iteself

#### Testing download of a single image

In [123]:
page = requests.get(url.image_page)
soup = bs4(page.content, 'html.parser')
uncleaned_image_link=(str(soup.find('link',href=re.compile(r'geekdo-images')))) #isolates the full link element to the image
dl=patt.findall(uncleaned_image_link) #removes everything except the link iteself
if dl !=[]:
    dl= dl[0]
#    url.direct_link=dl
    image = requests.get(dl)
    output_path = os.path.join(
        output_folder, url.game_name, url.image_number+dl[-4:]
    )
    print(output_path)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "wb") as f:
        f.write(image.content)
#else :
#    url.direct_link=uncleaned_image_link

boardgame-pictures/gloomhaven/2437871.jpg


Getting the correct output path

In [100]:
output_path = os.path.join(
    output_folder, url.game_name, url.image_number+'.jpg'
)
print(output_path)

boardgame-pictures/gloomhaven/2437871.jpg


Testing how iterating actually works

In [137]:
for idx_row in df2.head().iterrows():
    idx,row=idx_row
    print(idx_row)
    #print(idx)
    #print(row)

(0, image_number                                               2437871
game_name                                               gloomhaven
image_page      https://boardgamegeek.com/image/2437871/gloomhaven
direct_link                                                   None
Name: 0, dtype: object)
(1, image_number                                               2556210
game_name                                               gloomhaven
image_page      https://boardgamegeek.com/image/2556210/gloomhaven
direct_link                                                   None
Name: 1, dtype: object)
(2, image_number                                               2563554
game_name                                               gloomhaven
image_page      https://boardgamegeek.com/image/2563554/gloomhaven
direct_link                                                   None
Name: 2, dtype: object)
(3, image_number                                               2571642
game_name                                

In [147]:
df2.head()

,image_number,game_name,image_page,direct_link
0,2437871,gloomhaven,https://boardgamegeek.com/image/2437871/gloomhaven,None
1,2556210,gloomhaven,https://boardgamegeek.com/image/2556210/gloomhaven,None
2,2563554,gloomhaven,https://boardgamegeek.com/image/2563554/gloomhaven,None
3,2571642,gloomhaven,https://boardgamegeek.com/image/2571642/gloomhaven,None
4,2579136,gloomhaven,https://boardgamegeek.com/image/2579136/gloomhaven,None


In [146]:
df2.at[0,'direct_link']=None

In [150]:
df2head=pd.read_pickle('df_images_head')

In [151]:
df2head

,image_number,game_name,image_page,direct_link
0,2437871,gloomhaven,https://boardgamegeek.com/image/2437871/gloomhaven,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__imagepage/img/pBaOL7vV402nn1I5dHsdSKsFHqA=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2437871.jpg
1,2556210,gloomhaven,https://boardgamegeek.com/image/2556210/gloomhaven,https://cf.geekdo-images.com/8_t1fAfg2XUXZwXmQAKKEw__imagepage/img/YdUkrBtJyR5KChUOZRoNdbk6jfs=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2556210.jpg
2,2563554,gloomhaven,https://boardgamegeek.com/image/2563554/gloomhaven,https://cf.geekdo-images.com/KfNMyGdwipOJZymQep-nBw__imagepage/img/c9RLY7w-F2yqOBBprEa9cJyJWGk=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2563554.jpg
3,2571642,gloomhaven,https://boardgamegeek.com/image/2571642/gloomhaven,https://cf.geekdo-images.com/Q26KGdwoIk7vfe0vMUqAuA__imagepage/img/fE1XRGllK26lG7DNhXeL4HiTWOI=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2571642.jpg
4,2579136,gloomhaven,https://boardgamegeek.com/image/2579136/gloomhaven,https://cf.geekdo-images.com/R7CLLqDH_ymF_nOBkn1Qug__imagepage/img/jwYsOjFUYRbxHUmB1pxJY7k8IqY=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2579136.jpg


In [152]:
'https://cf.geekdo-images.com/eiZ1dJ8dND0UVpRT5ha0eA__original/img/kRVUqjNxKulURCmF53OWLL3jJ_U=/0x0/filters:format(png)/pic2602482.png'

'https://cf.geekdo-images.com/eiZ1dJ8dND0UVpRT5ha0eA__original/img/kRVUqjNxKulURCmF53OWLL3jJ_U=/0x0/filters:format(png)/pic2602482.png'

In [153]:
'https://cf.geekdo-images.com/R7CLLqDH_ymF_nOBkn1Qug__imagepage/img/jwYsOjFUYRbxHUmB1pxJY7k8IqY=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2579136.jpg'

'https://cf.geekdo-images.com/R7CLLqDH_ymF_nOBkn1Qug__imagepage/img/jwYsOjFUYRbxHUmB1pxJY7k8IqY=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2579136.jpg'

In [154]:
'https://cf.geekdo-images.com/R7CLLqDH_ymF_nOBkn1Qug__original/img/jwYsOjFUYRbxHUmB1pxJY7k8IqY=/0x0/filters:format(jpg)/pic2579136.jpg'

'https://cf.geekdo-images.com/R7CLLqDH_ymF_nOBkn1Qug__original/img/jwYsOjFUYRbxHUmB1pxJY7k8IqY=/0x0/filters:format(jpg)/pic2579136.jpg'

In [23]:
'https://cf.geekdo-images.com/R7CLLqDH_ymF_nOBkn1Qug__original/img/xsdFquOOjiolTa2QHZ6b5vMec4o=/0x0/filters:format(jpeg)/pic2579136.jpg'

'https://cf.geekdo-images.com/R7CLLqDH_ymF_nOBkn1Qug__original/img/xsdFquOOjiolTa2QHZ6b5vMec4o=/0x0/filters:format(jpeg)/pic2579136.jpg'

Had to repair a single file by hand

### Link Collector (OLD, using consecutive image number)

In [26]:
def get_links_simple(start,stop):
    linklist=[]
    errorlist=[]
    for image_number in range(start,stop): 
        if image_number%10==0 and image_number!=0:
                print(image_number)
        page = requests.get('https://boardgamegeek.com/image/'+str(image_number))
        if page.status_code==404:
            errorlist.append([image_number,404,None])#None for vectorization
        else:
            soup = bs4(page.content, 'html.parser')
            uncleaned_image_link=(str(soup.find('link',href=re.compile(r'geekdo-images')))) #isolates the full link element to the image
            patt= re.compile(r'https.*'+str(image_number)+r'[^"]*')  
            image_link=patt.findall(uncleaned_image_link) #removes everything except the link iteself

            uncleaned_game_name=(str(soup.find('link',rel='canonical'))) 
            patt2= re.compile(str(image_number)+r'/[^"]*')
            game_name=patt2.findall(uncleaned_game_name)
            
            if image_link==[] or game_name==[]:
                errorlist.append([image_number, page.status_code,None]) #None for vectorization
            else: 
                linklist.append([image_number,game_name[0][8:],image_link[0]])
            
    return [linklist, errorlist]

### collecting links (OLD, using consecutive image number)

In [40]:
l1 = get_links(2669798,2669806)

2669800


In [6]:
#l1

In [42]:
l2 = get_links(2669806,2669816)

In [7]:
#l2

In [10]:
l1=get_links_simple(2437871,2438871)

2437900
2438000
2438100
2438200
2438300
2438400
2438500
2438600
2438700
2438800


In [18]:
with open('game_links01', 'wb') as fp:
    pickle.dump(l1, fp)

In [12]:
with open ('game_links01', 'rb') as fp:
    l1_loaded = pickle.load(fp)

In [18]:
with open('testing', 'ab') as fp:
    pickle.dump([1,2,3,4,5], fp)

In [22]:
with open ('testing', 'rb') as fp:
    test_loaded = pickle.load(fp)

In [20]:
test_loaded

[1, 2, 3, 4]

In [136]:
l_save=[l2[0]+l3[0]+l4[0],l2[1]+l3[1]+l4[1]]

In [137]:
with open('game_links02', 'wb') as fp:
    pickle.dump(l_save, fp)

In [138]:
with open ('game_links02', 'rb') as fp:
    l_loaded = pickle.load(fp)

This took ~30 seconds

In [58]:
60*2*100

12000

So I could expect to get 12000 links in an hour

### some inspection

In [155]:
gns=pd.DataFrame(l_loaded[0])

In [156]:
gns

,0,1,2
0,2438871,entranced,https://cf.geekdo-images.com/U70FOKd_6j717C06XruJrw__imagepage/img/xgs8ngN6RvZfvop3stwcNGFHNck=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2438871.png
1,2438872,entranced,https://cf.geekdo-images.com/SukFOA6_sH7xcLo7NF1LzQ__imagepage/img/wMigVCnMrN97yWkaszA10G0H670=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2438872.png
2,2438873,entranced,https://cf.geekdo-images.com/wwlkbuFMPeGEQkPauyp-aA__imagepage/img/du5CWGW3Uwq4OqjIhcMUwu0VFdk=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2438873.png
3,2438874,entranced,https://cf.geekdo-images.com/7c_AiD--joSVIU9GyGgerA__imagepage/img/BToCrDQBpNJOL_YSKF3qcySFQ8I=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2438874.png
4,2438875,entranced,https://cf.geekdo-images.com/SzbQ0ZTlWuQh-f0TgRbTkQ__imagepage/img/xaYouPhVk-0ySBzZOB9Oxj_Ogzw=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2438875.png
...,...,...,...
832,2439566,murder-baldurs-gate-launch-weekend,https://cf.geekdo-images.com/wXxQmQwPpqQzreIGQdLypA__imagepage/img/Q0F2ceuSgzZwXAXprs3nhBEz6T8=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2439566.jpg
833,2439567,murder-baldurs-gate-launch-weekend,https://cf.geekdo-images.com/unUjmsWNqTUb9-2qfit-Jg__imagepage/img/en5R5Wt_GGGPLSPDakcmNOPuK8k=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2439567.jpg
834,2439568,murder-baldurs-gate-launch-weekend,https://cf.geekdo-images.com/iR0lfPph2RrmbNthTFRrhg__imagepage/img/5lejpE-Uq75ZcKR0ItMFaatYmL0=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2439568.jpg
835,2439569,murder-baldurs-gate-launch-weekend,https://cf.geekdo-images.com/PPsbTfuNUAmCfbGj9sSb8A__imagepage/img/4wnsg6oFFI-r3UhIceZ68Uoj250=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2439569.jpg


In [157]:
gns.columns=['image_id','game_title','image_link']

In [160]:
gns['game_title'].value_counts()

andrei123                          51
entranced                          45
                                   39
fogus                              30
mars-attacks-miniatures-game       30
                                   ..
cleric                              1
assassin                            1
jester                              1
mice-and-mystics                    1
fief-france-1429-buildings-pack     1
Name: game_title, Length: 190, dtype: int64

We need an effective way to isolate games from users

### Starting another test to use the gallery

In [161]:
page_t = requests.get('https://boardgamegeek.com/boardgame/174430/gloomhaven/images?pageid=1')
soup_t = bs4(page_t.content, 'html.parser')

In [162]:
page_t = requests.get('https://boardgamegeek.com/image/2437871/gloomhaven')
soup_t = bs4(page_t.content, 'html.parser')

In [163]:
soup_t.find_all("gg-app")

[<gg-app>
 <header class="skeleton-header">
 <nav>
 <span class="skeleton-header-logo"></span>
 <span class="skeleton-header-item"></span>
 <span class="skeleton-header-item"></span>
 <span class="skeleton-header-item"></span>
 <span class="skeleton-header-item"></span>
 <span class="skeleton-header-item"></span>
 <span class="skeleton-header-item"></span>
 </nav>
 </header>
 <main class="skeleton-main">
 <div class="loader">
 <div class="line"></div>
 <div class="line"></div>
 <div class="line"></div>
 <div class="line"></div>
 </div>
 </main>
 </gg-app>]

In [1]:
soup_t.find_all("script")

NameError: name 'soup_t' is not defined

### Independent approach: Download via google with specific queries limiting to bgg

For example, the following seem to offer decent results for a difficult game query.

intitle:"ark nova" site:boardgamegeek.com

"ark nova" site:boardgamegeek.com

ark-nova site:boardgamegeek.com

For the game "On Mars", the third option produced the best results. For "Beyond the sun", the second option seemed best.

In [165]:
path = Path('boardgame-pictures-google')

Uncleaned Searches is for a dictionary giving the correct board game names in the end

In [3]:
searches='Gloomhaven','Brass Birmingham','Pandemic Legacy Season 1','Ark Nova','Terraforming Mars','Gloomhaven Jaws of the Lion','Twilight Imperium Fourth Edition','Star Wars Rebellion','Gaia Project','War of the Ring Second Edition','Spirit Island','Through the Ages A New Story of Civilization','Great Western Trail','Twilight Struggle','Dune Imperium','Scythe','The Castles of Burgundy','Nemesis','7 Wonders Duel','Brass Lancashire','Concordia','A Feast for Odin','Terra Mystica','Wingspan','Arkham Horror The Card Game','Clank Legacy Acquisitions Incorporated','Eclipse Second Dawn for the Galaxy','Root','Everdell','Viticulture Essential Edition','Orleans','Lost Ruins of Arnak','Mage Knight Board Game','Food Chain Magnate','Barrage','Marvel Champions The Card Game','Puerto Rico','Too Many Bones','Caverna The Cave Farmers','Blood Rage','Pax Pamir Second Edition','Agricola','Underwater Cities','The Crew Mission Deep Sea','Anachrony','Maracaibo','Mansions of Madness Second Edition','Pandemic Legacy Season 2','The Crew The Quest for Planet Nine','On Mars','Tzolkin The Mayan Calendar','Power Grid','Clans of Caledonia','Crokinole','Le Havre','Star Wars Imperial Assault','Cascadia','Kingdom Death Monster','Pandemic Legacy Season 0','Mechs vs Minions','Lisboa','The Quacks of Quedlinburg','The Gallerist','Paladins of the West Kingdom','Azul','Android Netrunner','Eclipse','Aeons End','The 7th Continent','Through the Ages A Story of Civilization','Race for the Galaxy','Clank A DeckBuilding Adventure','Five Tribes','Fields of Arle','Teotihuacan City of Gods','Grand Austria Hotel','Agricola Revised Edition','Robinson Crusoe Adventures on the Cursed Island','Kanban EV','The Voyages of Marco Polo','7 Wonders','Lords of Waterdeep','Dominant Species','Sleeping Gods','Great Western Trail Second Edition','Tainted Grail The Fall of Avalon','Architects of the West Kingdom','El Grande','Keyflower','Caylus','Battlestar Galactica The Board Game','Beyond the Sun','Mombasa','Dominion Intrigue','Troyes','Raiders of the North Sea','The Lord of the Rings Journeys in MiddleEarth','Twilight Imperium Third Edition','Eldritch Horror','Lorenzo il Magnifico'

In [167]:
uncleaned_searches="Gloomhaven","Brass: Birmingham","Pandemic Legacy: Season 1","Ark Nova","Terraforming Mars","Gloomhaven: Jaws of the Lion","Twilight Imperium: Fourth Edition","Star Wars: Rebellion","Gaia Project","War of the Ring: Second Edition","Spirit Island","Through the Ages: A New Story of Civilization","Great Western Trail","Twilight Struggle","Dune: Imperium","Scythe","The Castles of Burgundy","Nemesis","7 Wonders Duel","Brass: Lancashire","Concordia","A Feast for Odin","Terra Mystica","Wingspan","Arkham Horror: The Card Game","Clank!: Legacy – Acquisitions Incorporated","Eclipse: Second Dawn for the Galaxy","Root","Everdell","Viticulture Essential Edition","Orléans","Lost Ruins of Arnak","Mage Knight Board Game","Food Chain Magnate","Barrage","Marvel Champions: The Card Game","Puerto Rico","Too Many Bones","Caverna: The Cave Farmers","Blood Rage","Pax Pamir: Second Edition","Agricola","Underwater Cities","The Crew: Mission Deep Sea","Anachrony","Maracaibo","Mansions of Madness: Second Edition","Pandemic Legacy: Season 2","The Crew: The Quest for Planet Nine","On Mars","Tzolk'in: The Mayan Calendar","Power Grid","Clans of Caledonia","Crokinole","Le Havre","Star Wars: Imperial Assault","Cascadia","Kingdom Death: Monster","Pandemic Legacy: Season 0","Mechs vs. Minions","Lisboa","The Quacks of Quedlinburg","The Gallerist","Paladins of the West Kingdom","Azul","Android: Netrunner","Eclipse","Aeon's End","The 7th Continent","Through the Ages: A Story of Civilization","Race for the Galaxy","Clank!: A Deck-Building Adventure","Five Tribes","Fields of Arle","Teotihuacan: City of Gods","Grand Austria Hotel","Agricola (Revised Edition)","Robinson Crusoe: Adventures on the Cursed Island","Kanban EV","The Voyages of Marco Polo","7 Wonders","Lords of Waterdeep","Dominant Species","Sleeping Gods","Great Western Trail (Second Edition)","Tainted Grail: The Fall of Avalon","Architects of the West Kingdom","El Grande","Keyflower","Caylus","Battlestar Galactica: The Board Game","Beyond the Sun","Mombasa","Dominion: Intrigue","Troyes","Raiders of the North Sea","The Lord of the Rings: Journeys in Middle-Earth","Twilight Imperium: Third Edition","Eldritch Horror","Lorenzo il Magnifico"

In [168]:
search_dict=dict(zip(searches,uncleaned_searches))
#search_dict

### Copied an attempt, didn't work

https://www.geeksforgeeks.org/how-to-download-google-images-using-python/?tab=article

### Another attempt using Selenium
Generated basic script in Selenium IDE, then iterated upon it

In [ ]:
# Generated by Selenium IDE
import pytest
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x55a783741463 <unknown>
#1 0x55a7835058d8 <unknown>
#2 0x55a78352b04a <unknown>
#3 0x55a783528c05 <unknown>
#4 0x55a78356c802 <unknown>
#5 0x55a78356c2af <unknown>
#6 0x55a783564443 <unknown>
#7 0x55a7835353c5 <unknown>
#8 0x55a783536531 <unknown>
#9 0x55a783793dce <unknown>
#10 0x55a783797192 <unknown>
#11 0x55a78377893e <unknown>
#12 0x55a783798103 <unknown>
#13 0x55a78376bd85 <unknown>
#14 0x55a7837b90a8 <unknown>
#15 0x55a7837b9239 <unknown>
#16 0x55a7837d4492 <unknown>
#17 0x7fbdcab64609 start_thread


In [10]:
browser = webdriver.Chrome("/home/jonas/programs/chromedriver")
browser.get('http://selenium.dev/')

/tmp/ipykernel_1460/1956462404.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("/home/jonas/programs/chromedriver")


WebDriverException: Message: Service /home/jonas/programs/chromedriver unexpectedly exited. Status code was: 127


In [8]:
browser = webdriver.Chrome('../../programs/chromedriver')
browser.get('http://selenium.dev/')

/tmp/ipykernel_1460/1543986041.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('../../programs/chromedriver')


AttributeError: 'Service' object has no attribute 'process'

In [ ]:

class TestGloomhaven():
  def setup_method(self, method):
    self.driver = webdriver.Chrome()
    self.vars = {}
    self.link_list = []

  def teardown_method(self, method):
    self.driver.quit()
  
  def test_gloomhaven(self):
    # Test name: Gloomhaven
    # Step # | name | target | value
    # 1 | open | https://boardgamegeek.com/image/2437871/gloomhaven | 
    self.driver.get("https://boardgamegeek.com/image/2437871/gloomhaven")
    # 2 | click | css=.tw-right-0 > .svg-inline--fa > path | 
    #self.driver.find_element(By.CSS_SELECTOR, ".tw-right-0 > .svg-inline--fa > path").click()
    # 3 | executeScript | return window.location.href | pageurl
    #self.vars["pageurl"] = self.driver.execute_script("return window.location.href")
    for j in range(10):
        self.driver.find_element(By.CSS_SELECTOR, ".tw-right-0 > .svg-inline--fa > path").click()
        self.link_list.append(self.driver.execute_script("return window.location.href"))
    return self.link_list
    
  


In [19]:
tester=TestGloomhaven()

In [20]:
tester.setup_method()

TypeError: TestGloomhaven.setup_method() missing 1 required positional argument: 'method'

In [21]:
tester.test_gloomhaven()

AttributeError: 'TestGloomhaven' object has no attribute 'driver'

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home
